In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [2]:
# Supposons que df_players est votre DataFrame préparé
# Il doit contenir les colonnes : 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent'

# Exemple initialisation ou import réel:
# df_players = pd.read_csv('votre_fichier.csv')
# df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(float)

budget = 1500  # Exemple budget total (en 0.1M £)
alpha = 0    # Pondération ownership (à ajuster: plus grand = plus pénalisant)

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Créer problème d'optimisation
prob = LpProblem("FPL_Team_Optimizer", LpMaximize)

# Variables binaires pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : maximiser points_per_cost tout en minimisant selected_by_percent
prob += lpSum([
    df_players.loc[i, 'points_per_cost'] * player_vars[i] - 
    alpha * df_players.loc[i, 'selected_by_percent'] * player_vars[i]
    for i in range(len(df_players))
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte : max 3 joueurs par équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i]
        for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Contraintes budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= budget

# Contrainte minimum minutes (exemple)
prob += lpSum([df_players.loc[i, 'minutes'] * player_vars[i] for i in range(len(df_players))]) >= 1000

# Résoudre
prob.solve()

# Extraire liste des joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

# Afficher résultats clés
print(selected_df[['web_name', 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent']])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/af85476201ea4ab49b8449b9a0e632da-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/af85476201ea4ab49b8449b9a0e632da-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 5325 RHS
At line 5353 BOUNDS
At line 6096 ENDATA
Problem MODEL has 27 rows, 742 columns and 3391 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10.408 - 0.00 seconds
Cgl0004I processed model has 27 rows, 624 columns (624 integer (552 of which binary)) and 2919 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 3.33067e-16
Cbc0038I Solution found of -10.408
Cbc0038I Cleaned solution of -10.408
Cbc0038I Before mini branch and bound, 624 integers at bound fixed and 0 continuous

In [3]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,clean_sheets_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
0,Omar,Alderete,SUN,Sunderland,DEF,41.0,39.0,0.951220,0.080082,3.000000,29.8,6.1,63,7.5,487.0,673728,2200,83384,53,1.0,1.0,2.0,0.58,0.33,0.91,2.197802,0.11,0.00,0.06,0.17,1.22,0.74,10.90,0.37,67.50,0.1625,0.1175,4.625,7.50,46.153846
1,Marcos,Senesi,BOU,Bournemouth,DEF,49.0,45.0,0.918367,0.078809,3.166667,28.1,24.0,14,5.0,571.0,2853986,3437,196153,89,2.0,0.0,2.0,0.14,0.41,0.55,3.636364,0.02,0.00,0.06,0.08,0.87,1.10,13.08,0.47,76.00,0.0800,0.0000,3.900,5.00,62.500000
2,Marc,Guéhi,CRY,Crystal Palace,DEF,48.0,43.0,0.895833,0.079630,3.166667,23.1,29.9,9,5.0,540.0,2461413,2340,442180,82,2.0,1.0,3.0,0.39,0.09,0.48,6.250000,0.06,0.00,0.02,0.08,1.14,0.50,7.83,0.50,67.50,0.0750,0.0575,2.875,5.00,66.666667
4,Tino,Livramento,NEW,Newcastle,DEF,51.0,37.0,0.725490,0.070342,3.000000,14.3,7.2,56,4.5,526.0,935222,34,577056,3042,1.0,0.0,1.0,0.00,0.33,0.33,3.030303,0.00,0.00,0.06,0.06,0.73,0.51,3.08,0.86,64.00,0.0175,0.0000,1.025,4.50,257.142857
5,Joe,Rodon,LEE,Leeds,DEF,40.0,29.0,0.725000,0.053704,2.666667,20.7,5.2,73,3.2,540.0,528761,893,170185,65,0.0,1.0,1.0,0.33,0.01,0.34,2.941176,0.06,0.00,0.00,0.06,1.22,1.50,9.33,0.33,67.50,0.0850,0.0825,3.625,3.25,38.235294
42,Igor Thiago,Thiago,BRE,Brentford,FWD,60.0,30.0,0.500000,0.060729,3.166667,35.2,2.7,112,4.0,494.0,360429,766,179514,94,0.0,4.0,4.0,2.05,0.16,2.21,1.809955,0.37,0.00,0.03,0.40,1.52,1.82,5.10,0.18,56.00,0.2100,0.2000,4.125,4.00,19.047619
50,Wilson,Isidor,SUN,Sunderland,FWD,56.0,27.0,0.482143,0.084639,3.000000,25.0,3.2,104,3.2,319.0,447106,391,118813,142,0.0,3.0,3.0,1.23,0.03,1.26,2.380952,0.35,0.00,0.01,0.36,1.37,0.85,3.67,0.56,63.25,0.2450,0.2375,3.025,3.25,13.265306
51,João Pedro,João Pedro,CHE,Chelsea,FWD,77.0,37.0,0.480519,0.072549,2.666667,40.2,62.7,1,2.8,510.0,2758918,866,1755992,1935,3.0,2.0,5.0,1.50,0.23,1.73,2.890173,0.26,0.00,0.04,0.30,1.27,1.24,4.94,0.35,64.50,0.0575,0.0375,3.000,2.75,47.826087
3,Robin,Roefs,SUN,Sunderland,GKP,46.0,39.0,0.847826,0.072222,3.000000,19.9,4.3,84,5.5,540.0,508144,901,103453,36,0.0,0.0,0.0,0.00,0.01,0.01,0.000000,0.00,3.33,0.00,0.00,1.15,0.67,0.00,0.50,67.50,0.0025,0.0000,2.950,5.50,2200.000000
11,Nick,Pope,NEW,Newcastle,GKP,51.0,34.0,0.666667,0.062963,3.000000,13.5,8.7,44,4.0,540.0,728005,1270,222975,132,1.0,0.0,1.0,0.00,0.00,0.00,0.000000,0.00,2.50,0.00,0.00,0.78,0.83,0.00,0.67,67.50,0.0000,0.0000,2.200,4.00,0.000000


In [4]:
selected_df['price'].sum()

np.float64(810.0)

In [5]:
selected_df['selected_by_percent'].sum()

np.float64(225.89999999999998)